#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/t/ab8726f0cfc84544abbae69a6be88071

## Librerias

In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder,StandardScaler,MinMaxScaler,LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris, fetch_california_housing

import urllib.request
from PIL import Image

## Data

In [2]:
### Para que funcione necesitas bajarte los archivos de datos de Kaggle 
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")



## Data exploration

### TEST


In [3]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         391 non-null    int64  
 1   Company           391 non-null    object 
 2   Product           391 non-null    object 
 3   TypeName          391 non-null    object 
 4   Inches            391 non-null    float64
 5   ScreenResolution  391 non-null    object 
 6   Cpu               391 non-null    object 
 7   Ram               391 non-null    object 
 8   Memory            391 non-null    object 
 9   Gpu               391 non-null    object 
 10  OpSys             391 non-null    object 
 11  Weight            391 non-null    object 
dtypes: float64(1), int64(1), object(10)
memory usage: 36.8+ KB


In [104]:
test

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Ram_GB,Gpu,OpSys,WeightInKgs,Capacidad_GB,Cpu_brand,CpuType
0,750,Toshiba,Tecra X40-D-10G,Notebook,14.0,1920x1080,8,Intel HD Graphics 620,Windows 10,1.25,256,Intel,Core i5 7200U 2.5GHz
1,726,Mediacom,Smartbook 142,Notebook,14.0,1920x1200,4,Intel HD Graphics 400,Windows 10,1.40,32,Intel,Atom Z8350 1.92GHz
2,633,Mediacom,SmartBook 140,Notebook,14.0,1920x1080,2,Intel HD Graphics,Windows 10,1.40,32,Intel,Atom x5-Z8350 1.44GHz
3,363,Dell,Inspiron 7577,Gaming,15.6,1920x1080,8,Nvidia GeForce GTX 1050,Windows 10,2.65,1000,Intel,Core i5 7300HQ 2.5GHz
4,319,Asus,VivoBook Flip,2 in 1 Convertible,11.6,1366x768,2,Intel HD Graphics 500,Windows 10,1.10,32,Intel,Celeron Dual Core N3350 1.1GHz
...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,693,Dell,Alienware 15,Gaming,15.6,1920x1080,16,Nvidia GeForce GTX 1070,Windows 10,3.49,256,Intel,Core i7 7820HK 2.9GHz
387,1132,Razer,Blade Stealth,Ultrabook,12.5,3840x2160,8,Intel HD Graphics 520,Windows 10,1.25,256,Intel,Core i7 6500U 2.5GHz
388,49,Dell,Inspiron 3567,Notebook,15.6,1920x1080,4,AMD Radeon R5 M430,Windows 10,2.30,256,Intel,Core i5 7200U 2.5GHz
389,1243,Asus,X540SA-RBPDN09 (N3710/4GB/1TB/W10),Notebook,15.6,1366x768,4,Intel HD Graphics 405,Windows 10,2.65,1000,Intel,Pentium Quad Core N3710 1.6GHz


In [60]:
test.rename(columns={'Weight': 'WeightInKgs'}, inplace=True)
test['WeightInKgs'] = test['WeightInKgs'].str.rstrip('kg')
test['WeightInKgs'] = test['WeightInKgs'].astype(float)

In [61]:
test['Ram'] = test['Ram'].str.rstrip('GB')
test['Ram'] = test['Ram'].astype(int)
test.rename(columns={'Ram': 'Ram_GB'}, inplace=True)

In [62]:
test = pd.DataFrame(test)

# Regular expression pattern para extraer números antes de GB y TB
pattern = r'(\d+)GB|\s*(\d+)TB'

# Aplica el patrón y crea una nueva columna 'Capacidad_GB' con los números antes de GB
test['Capacidad_GB'] = test['Memory'].str.extract(pattern)[0]

# Aplica el patrón y crea una nueva columna 'Capacidad_TB' con los números antes de TB
test['Capacidad_TB'] = test['Memory'].str.extract(pattern)[1]


In [63]:
test['Capacidad_TB'].fillna(0, inplace=True)
test['Capacidad_TB'] = test['Capacidad_TB'].astype(int)
test['Capacidad_TB'] = test['Capacidad_TB'].apply(lambda x: x*1000)

test['Capacidad_GB'].fillna(0, inplace=True)
test['Capacidad_GB'] = test['Capacidad_GB'].astype(int)
test['Capacidad_GB'] = test['Capacidad_GB']+test['Capacidad_TB']
test[test['Capacidad_GB']== 0] = 1000
columns_delete = ['Memory','Capacidad_TB']
test = test.drop(columns_delete, axis=1)

In [64]:
test[['ScreenType', 'ScreenResolution']] = test['ScreenResolution'].str.rsplit(' ', n=1, expand=True)
test['ScreenResolution'] = test['ScreenResolution'].fillna(test['ScreenType'])

In [71]:
test = test.drop('ScreenType', axis = 1)

In [65]:
test[['Cpu_brand', 'CpuType']] = test['Cpu'].str.split(' ', n=1, expand=True)
test = test.drop('Cpu', axis= 1)

### TRAIN
Hacemos las modifciaciones en train

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         912 non-null    int64  
 1   Company           912 non-null    object 
 2   Product           912 non-null    object 
 3   TypeName          912 non-null    object 
 4   Inches            912 non-null    float64
 5   ScreenResolution  912 non-null    object 
 6   Cpu               912 non-null    object 
 7   Ram               912 non-null    object 
 8   Memory            912 non-null    object 
 9   Gpu               912 non-null    object 
 10  OpSys             912 non-null    object 
 11  Weight            912 non-null    object 
 12  Price_euros       912 non-null    float64
dtypes: float64(2), int64(1), object(10)
memory usage: 92.8+ KB


In [5]:
train.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1104,Acer,Aspire ES1-523,Notebook,15.6,1366x768,AMD A8-Series 7410 2.2GHz,4GB,500GB HDD,AMD Radeon R5,Windows 10,2.4kg,387.0
1,114,Dell,XPS 13,Ultrabook,13.3,Quad HD+ / Touchscreen 3200x1800,Intel Core i7 7560U 2.4GHz,8GB,256GB SSD,Intel Iris Plus Graphics 640,Windows 10,1.23kg,1379.0
2,172,Acer,Aspire A517-51G,Notebook,17.3,IPS Panel Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,Nvidia GeForce MX150,Windows 10,3kg,854.0
3,918,HP,Elitebook 820,Netbook,12.5,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,512GB SSD,Intel HD Graphics 620,Windows 10,1.26kg,1483.0
4,447,Lenovo,IdeaPad 320-15AST,Notebook,17.3,1600x900,AMD A6-Series 9220 2.5GHz,8GB,1TB HDD,AMD Radeon R4,Windows 10,2.8kg,519.0


In [6]:
print(train['Company'].unique())

print(train['TypeName'].unique())  #convertir

print(train['ScreenResolution'].unique())

print(train['Ram_GB'].unique())

print(train['Memory'].unique())

print(train['OpSys'].unique())    #convertir


['Acer' 'Dell' 'HP' 'Lenovo' 'Asus' 'Xiaomi' 'Apple' 'Samsung' 'Razer'
 'Toshiba' 'Fujitsu' 'Microsoft' 'MSI' 'LG' 'Mediacom' 'Google' 'Vero'
 'Huawei' 'Chuwi']
['Notebook' 'Ultrabook' 'Netbook' 'Gaming' '2 in 1 Convertible'
 'Workstation']
['1366x768' 'Quad HD+ / Touchscreen 3200x1800'
 'IPS Panel Full HD 1920x1080' 'Full HD 1920x1080' '1600x900'
 'IPS Panel Full HD / Touchscreen 1920x1080'
 'IPS Panel Retina Display 2304x1440'
 '4K Ultra HD / Touchscreen 3840x2160' 'Touchscreen 2560x1440'
 'Full HD / Touchscreen 1920x1080' '1440x900'
 'IPS Panel Quad HD+ / Touchscreen 3200x1800' 'Touchscreen 2256x1504'
 'IPS Panel 4K Ultra HD / Touchscreen 3840x2160'
 'IPS Panel Touchscreen / 4K Ultra HD 3840x2160' 'IPS Panel 1366x768'
 '4K Ultra HD 3840x2160' 'IPS Panel 4K Ultra HD 3840x2160'
 'IPS Panel Retina Display 2880x1800' 'IPS Panel Touchscreen 2560x1440'
 'Touchscreen 1366x768' '2560x1440' 'IPS Panel Retina Display 2560x1600'
 'Quad HD+ 3200x1800' 'IPS Panel Full HD 2560x1440' '1920x1080'
 

KeyError: 'Ram_GB'

In [8]:
train.rename(columns={'Weight': 'WeightInKgs'}, inplace=True)
train['WeightInKgs'] = train['WeightInKgs'].str.rstrip('kg')
train['WeightInKgs'] = train['WeightInKgs'].astype(float)

In [9]:
train['Ram'] = train['Ram'].str.rstrip('GB')
# train['Ram'] = train['Ram'].map(lambda x : x.str.rstrip('GB'))  es lo mismo. Se podria crear una columna nueva ram_l x ej para qeu se vean estos datos, pero luego habria que drop las antiguas para no tener columnas duplicadas
train['Ram'] = train['Ram'].astype(int)
train.rename(columns={'Ram': 'Ram_GB'}, inplace=True)

**Para columnas categoricas**


**Memory** pasos a seguir:
- Separar numerico con tb o gb
- transformar tb a gb
- eliminar gb
- separar 

In [10]:
train = pd.DataFrame(train)

# Regular expression pattern para extraer números antes de GB y TB
pattern = r'(\d+)GB|\s*(\d+)TB'

# Aplica el patrón y crea una nueva columna 'Capacidad_GB' con los números antes de GB
train['Capacidad_GB'] = train['Memory'].str.extract(pattern)[0]

# Aplica el patrón y crea una nueva columna 'Capacidad_TB' con los números antes de TB
train['Capacidad_TB'] = train['Memory'].str.extract(pattern)[1]


In [11]:
train['Capacidad_TB'].fillna(0, inplace=True)
train['Capacidad_TB'] = train['Capacidad_TB'].astype(int)
train['Capacidad_TB'] = train['Capacidad_TB'].apply(lambda x: x*1000)

train['Capacidad_GB'].fillna(0, inplace=True)
train['Capacidad_GB'] = train['Capacidad_GB'].astype(int)
train['Capacidad_GB'] = train['Capacidad_GB']+train['Capacidad_TB']
train[train['Capacidad_GB']== 0] = 1000



In [13]:
train.shape


(912, 15)

In [14]:
train['Capacidad_GB'].value_counts()

Capacidad_GB
256     357
1000    173
128     130
512      98
500      95
32       29
64       11
2000      9
16        7
180       2
508       1
Name: count, dtype: int64

In [45]:
columns_delete = ['Memory','Capacidad_TB']
train = train.drop(columns_delete, axis=1)

**Screen**
- dividir entre lo que hay delante de la resolucion y convertir a numero.

In [40]:
train[['ScreenType', 'ScreenResolution']] = train['ScreenResolution'].str.rsplit(' ', n=1, expand=True)
train['ScreenResolution'] = train['ScreenResolution'].fillna(train['ScreenType'])


ValueError: Columns must be same length as key

In [42]:
train = train.drop('ScreenType', axis = 1)

**CPU**

In [56]:
train[['Cpu_brand', 'CpuType']] = train['Cpu'].str.split(' ', n=1, expand=True)
train = train.drop('Cpu', axis= 1)

### OneHotEncoder

Aplicamos la función para los dos df TRAIN y TEST

In [58]:
# def apply_onehot_encoder(train:pd.DataFrame, columns_to_encode:list, test:pd.DataFrame=None):
    
#     # Resetear índices para evitar desalineación
#     train = train.reset_index(drop=True)
    
#     # Crear el OneHotEncoder
#     encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

#     # Ajustar y transformar las columnas seleccionadas
#     transformed_data = encoder.fit_transform(train[columns_to_encode])

#     # Crear un DataFrame con las columnas transformadas
#     transformed_df = pd.DataFrame(transformed_data, columns=encoder.get_feature_names_out(columns_to_encode))
    
#     # Concatenar con el DataFrame original excluyendo las columnas transformadas
#     df_concatenated = pd.concat([train.drop(columns_to_encode, axis=1), transformed_df], axis=1)

#     # Si se proporciona un segundo DataFrame, aplicar la misma transformación
#     if test is not None:
#         transformed_data_to_transform = encoder.transform(test[columns_to_encode])
#         transformed_df_to_transform = pd.DataFrame(transformed_data_to_transform, columns=encoder.get_feature_names_out(columns_to_encode))
#         df_to_transform_concatenated = pd.concat([test.drop(columns_to_encode, axis=1), transformed_df_to_transform], axis=1)
#         return df_concatenated, df_to_transform_concatenated

#     return df_concatenated

Me daba error de tipo, asi que he añadido una linea a la función de juan, para que se todo se transforme a strings. ERA PORQUE HAY NANS

In [82]:
def apply_onehot_encoder(train: pd.DataFrame, columns_to_encode: list, test: pd.DataFrame = None):
    
    # Resetear índices para evitar desalineación
    train = train.reset_index(drop=True)
    
    # Convertir todas las columnas a strings
    train[columns_to_encode] = train[columns_to_encode].astype(str)

    # Crear el OneHotEncoder
    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

    # Ajustar y transformar las columnas seleccionadas
    transformed_data = encoder.fit_transform(train[columns_to_encode])

    # Crear un DataFrame con las columnas transformadas
    transformed_df = pd.DataFrame(transformed_data, columns=encoder.get_feature_names_out(columns_to_encode))
    
    # Concatenar con el DataFrame original excluyendo las columnas transformadas
    df_concatenated = pd.concat([train.drop(columns=columns_to_encode), transformed_df], axis=1)

    # Si se proporciona un segundo DataFrame, aplicar la misma transformación
    if test is not None:
        # Convertir todas las columnas a strings en el conjunto de prueba
        test[columns_to_encode] = test[columns_to_encode].astype(str)
        
        transformed_data_to_transform = encoder.transform(test[columns_to_encode])
        transformed_df_to_transform = pd.DataFrame(transformed_data_to_transform, columns=encoder.get_feature_names_out(columns_to_encode))
        df_to_transform_concatenated = pd.concat([test.drop(columns=columns_to_encode), transformed_df_to_transform], axis=1)
        return df_concatenated, df_to_transform_concatenated

    return df_concatenated

In [86]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         912 non-null    int64  
 1   Company           912 non-null    object 
 2   Product           912 non-null    object 
 3   TypeName          912 non-null    object 
 4   Inches            912 non-null    float64
 5   ScreenResolution  904 non-null    object 
 6   Ram_GB            912 non-null    int32  
 7   Gpu               912 non-null    object 
 8   OpSys             912 non-null    object 
 9   WeightInKgs       912 non-null    float64
 10  Price_euros       912 non-null    float64
 11  Capacidad_GB      912 non-null    int64  
 12  Cpu_brand         904 non-null    object 
 13  CpuType           904 non-null    object 
dtypes: float64(3), int32(1), int64(2), object(8)
memory usage: 96.3+ KB


In [90]:
columns_to_encode = ['Company','TypeName','Cpu_brand','ScreenResolution','OpSys']
train_ampliado, test_ampliado = apply_onehot_encoder(train,columns_to_encode,test)

c:\Users\laura\OneDrive\Desktop\DS_PT_09_2023\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [91]:
train_ampliado.shape

(912, 64)

## Data processing

Utilizariamos una transformación logaritmica en los datos de precio para normalizar la curva: 
***La transformación logarítmica se usa normalmente si los datos tienen una distribución sesgada de forma positiva (como se muestra a continuación) y algunos valores son muy grandes. Si estos valores grandes están en su área de estudio, la transformación logarítmica le ayudará a que las varianzas sean más constantes y normalizará sus datos. Con respecto a la terminología, cuando se implementa una transformación logarítmica con kriging, el método de predicción se conoce como kriging lognormal, mientras que para todos los demás valores de λ, el método de kriging asociado se conoce como kriging transgaussiano.***



In [92]:
train_ampliado['Price_euros'] = np.log(train_ampliado['Price_euros'])

-----------------------------------------------------------------------------------------------------------------

## Pasos que si o si debéis realizar para poder participar en la competición:

In [93]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler,MinMaxScaler,LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

### 1. Definir X e y

In [121]:
X = train_ampliado.drop(['Price_euros', 'laptop_ID', 'Product','Gpu','CpuType'], axis=1)
y = train_ampliado['Price_euros'].copy()
X.shape

(912, 59)

In [122]:
y.shape

(912,)

In [123]:
X.columns

Index(['Inches', 'Ram_GB', 'WeightInKgs', 'Capacidad_GB', 'Company_1000',
       'Company_Acer', 'Company_Apple', 'Company_Asus', 'Company_Chuwi',
       'Company_Dell', 'Company_Fujitsu', 'Company_Google', 'Company_HP',
       'Company_Huawei', 'Company_LG', 'Company_Lenovo', 'Company_MSI',
       'Company_Mediacom', 'Company_Microsoft', 'Company_Razer',
       'Company_Samsung', 'Company_Toshiba', 'Company_Vero', 'Company_Xiaomi',
       'TypeName_1000', 'TypeName_2 in 1 Convertible', 'TypeName_Gaming',
       'TypeName_Netbook', 'TypeName_Notebook', 'TypeName_Ultrabook',
       'TypeName_Workstation', 'Cpu_brand_AMD', 'Cpu_brand_Intel',
       'Cpu_brand_nan', 'ScreenResolution_1366x768',
       'ScreenResolution_1440x900', 'ScreenResolution_1600x900',
       'ScreenResolution_1920x1080', 'ScreenResolution_1920x1200',
       'ScreenResolution_2160x1440', 'ScreenResolution_2256x1504',
       'ScreenResolution_2304x1440', 'ScreenResolution_2400x1600',
       'ScreenResolution_2560x144

In [124]:
numeric_features = ['Inches','Ram_GB', 'WeightInKgs','Capacidad_GB']      

In [125]:
scaler = StandardScaler()

# Ajusta y transforma las columnas seleccionadas en el conjunto de entrenamiento (X)
X[numeric_features] = scaler.fit_transform(X[numeric_features])

### 2. Dividir X_train, X_test, y_train, y_test

In [126]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42) 

### 3. Asignar el modelo (vacío) a una variable


In [127]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
# probamos OTROS MODELOS, AHORA EL RIDGE/SVR
from sklearn.linear_model import Ridge, Lasso
# model = Ridge()

from sklearn.svm import SVR
# model = SVR(kernel= 'Poly', degree=3)
# el kernel puede ser linear, poly....


In [133]:
# model =  LinearRegression()
# model = Ridge()
model = SVR()
model

SVR()

In [134]:
# Esto lo hacemos antes del fit para comprobar cuál sería mejor.

scores = -cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_error')
print(scores)
np.mean(scores)
# para hacer cross validation con los modelos que hemos creado. 
# Tenemos scoring podemos elegir cuál queremmos. Como queremos el RMSE 

[0.28464483 0.30547203 0.30055097 0.3347987  0.33030393]


0.31115409160800184

In [135]:
# ¿cuánto nos estamos equivocando? Si el RMSE es de 300, y el precio es de 191, es mucho...
print(y.min())
print(y.max())

5.256974403013654
8.612321536507814


In [136]:
model.fit(X_test,y_test)

SVR()

In [137]:
pred = model.predict(X_test)
pred

array([6.96981522, 5.77002132, 6.96986533, 7.14713499, 7.31222014,
       6.07118193, 7.10045446, 7.30037376, 6.96986533, 7.21264533,
       6.94670521, 6.58833686, 5.80305602, 7.28840075, 7.30047011,
       7.32368228, 7.31622597, 7.16061829, 6.7523544 , 6.49644594,
       6.34494994, 6.20706439, 6.59836919, 6.46453262, 6.83571643,
       6.56759754, 7.00140263, 7.32403327, 6.6530588 , 6.4497927 ,
       7.4165817 , 7.08191473, 6.25644877, 7.15212402, 6.95784594,
       6.22221041, 6.18702434, 6.36261998, 7.28887495, 6.80744911,
       7.03243082, 6.90079301, 6.95832739, 7.55100333, 6.70113936,
       5.99800277, 6.85557029, 6.39143446, 7.13558041, 6.18319116,
       7.34725359, 6.58369677, 6.05081433, 7.49707662, 7.06212532,
       7.43366162, 7.05884676, 7.03228397, 7.65060467, 7.32360537,
       6.02602481, 6.95843725, 6.01889567, 7.11990916, 6.04648805,
       6.60554017, 6.1078244 , 6.7078446 , 7.3374549 , 6.18068216,
       7.0751209 , 6.52807186, 7.7592353 , 7.11990916, 7.16330

In [139]:
model.fit(X,y)

SVR()

In [140]:
pred = model.predict(X)
pred.shape

(912,)

### 4. Sacar métricas, valorar el modelo 

Recuerda que en la competición se va a evaluar con la métrica de MAE.

In [141]:
from sklearn.metrics import mean_squared_error
import numpy as np

print('RMSE :',np.sqrt(mean_squared_error(y,pred)))

RMSE : 0.27187378951520413


-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `test.csv` nos arriesgamos a que digamos, no salga lo esperado.

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `train.csv` al completo.


**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

### 2. Carga los datos de `test.csv` para predecir.

In [147]:
test_ampliado.columns

Index(['laptop_ID', 'Product', 'Inches', 'Ram_GB', 'Gpu', 'WeightInKgs',
       'Capacidad_GB', 'CpuType', 'Company_1000', 'Company_Acer',
       'Company_Apple', 'Company_Asus', 'Company_Chuwi', 'Company_Dell',
       'Company_Fujitsu', 'Company_Google', 'Company_HP', 'Company_Huawei',
       'Company_LG', 'Company_Lenovo', 'Company_MSI', 'Company_Mediacom',
       'Company_Microsoft', 'Company_Razer', 'Company_Samsung',
       'Company_Toshiba', 'Company_Vero', 'Company_Xiaomi', 'TypeName_1000',
       'TypeName_2 in 1 Convertible', 'TypeName_Gaming', 'TypeName_Netbook',
       'TypeName_Notebook', 'TypeName_Ultrabook', 'TypeName_Workstation',
       'Cpu_brand_AMD', 'Cpu_brand_Intel', 'Cpu_brand_nan',
       'ScreenResolution_1366x768', 'ScreenResolution_1440x900',
       'ScreenResolution_1600x900', 'ScreenResolution_1920x1080',
       'ScreenResolution_1920x1200', 'ScreenResolution_2160x1440',
       'ScreenResolution_2256x1504', 'ScreenResolution_2304x1440',
       'ScreenResolut

In [144]:
X1 = test_ampliado.drop(['laptop_ID', 'Product','Gpu','CpuType'], axis=1)

numeric_features = ['Inches','Ram_GB', 'WeightInKgs','Capacidad_GB']    
scaler = StandardScaler()

# Ajusta y transforma las columnas seleccionadas en el conjunto de entrenamiento (X)
X1[numeric_features] = scaler.fit_transform(X[numeric_features])

In [145]:
X1.shape

(391, 59)

In [146]:
model

SVR()

In [148]:
final_predict = model.predict(X1)

In [149]:
final_predict

array([7.21770295, 5.97317854, 5.97210772, 7.08364998, 6.05056755,
       6.87972685, 6.50209406, 7.34302228, 7.39717623, 7.08908423,
       6.38399317, 6.94803682, 7.28727668, 6.23885103, 6.75760631,
       6.94851772, 7.19542131, 7.71551889, 6.4178593 , 6.33522011,
       7.13010078, 6.41142886, 6.54435359, 6.56593669, 6.0366865 ,
       7.67656712, 6.39234471, 6.30827631, 6.36689858, 7.08908423,
       6.84473359, 6.90516194, 6.88953418, 6.68323092, 7.23056388,
       7.21741045, 6.84472252, 7.19523898, 6.25890906, 7.6537825 ,
       7.58716977, 7.08908423, 5.71468798, 7.35845169, 6.9486449 ,
       7.59820223, 6.93967591, 7.58644886, 6.94924841, 7.15255799,
       7.08549313, 6.47502298, 6.65906427, 7.1663637 , 5.87948156,
       6.87768238, 6.81631511, 6.04928311, 6.94771226, 7.2176242 ,
       6.61427325, 7.57168503, 6.76656132, 6.23255435, 6.77662643,
       7.18201412, 6.78189108, 7.5299813 , 6.65903808, 6.93395891,
       5.8793463 , 7.35845169, 7.41504061, 7.14609464, 6.40335

In [167]:
predictions_submit_reshaped = final_predict.reshape(-1, 1)
predictions_submit_reshaped


array([[7.21770295],
       [5.97317854],
       [5.97210772],
       [7.08364998],
       [6.05056755],
       [6.87972685],
       [6.50209406],
       [7.34302228],
       [7.39717623],
       [7.08908423],
       [6.38399317],
       [6.94803682],
       [7.28727668],
       [6.23885103],
       [6.75760631],
       [6.94851772],
       [7.19542131],
       [7.71551889],
       [6.4178593 ],
       [6.33522011],
       [7.13010078],
       [6.41142886],
       [6.54435359],
       [6.56593669],
       [6.0366865 ],
       [7.67656712],
       [6.39234471],
       [6.30827631],
       [6.36689858],
       [7.08908423],
       [6.84473359],
       [6.90516194],
       [6.88953418],
       [6.68323092],
       [7.23056388],
       [7.21741045],
       [6.84472252],
       [7.19523898],
       [6.25890906],
       [7.6537825 ],
       [7.58716977],
       [7.08908423],
       [5.71468798],
       [7.35845169],
       [6.9486449 ],
       [7.59820223],
       [6.93967591],
       [7.586

In [169]:

# Reverse the scaling on the predictions using inverse_transform

scaler=StandardScaler()
scaler.fit(predictions_submit_reshaped)
predictions_submit_inverse = scaler.inverse_transform(predictions_submit_reshaped)


In [171]:
predictions_submit_inverse.shape

(391, 1)

In [ ]:

# If needed, reshape the result back to a 1D array
predictions_submit_inverse = predictions_submit_inverse.flatten()

**¿Por qué da error?** 

In [150]:
print(final_predict.shape)
print(final_predict)

(391,)
[7.21770295 5.97317854 5.97210772 7.08364998 6.05056755 6.87972685
 6.50209406 7.34302228 7.39717623 7.08908423 6.38399317 6.94803682
 7.28727668 6.23885103 6.75760631 6.94851772 7.19542131 7.71551889
 6.4178593  6.33522011 7.13010078 6.41142886 6.54435359 6.56593669
 6.0366865  7.67656712 6.39234471 6.30827631 6.36689858 7.08908423
 6.84473359 6.90516194 6.88953418 6.68323092 7.23056388 7.21741045
 6.84472252 7.19523898 6.25890906 7.6537825  7.58716977 7.08908423
 5.71468798 7.35845169 6.9486449  7.59820223 6.93967591 7.58644886
 6.94924841 7.15255799 7.08549313 6.47502298 6.65906427 7.1663637
 5.87948156 6.87768238 6.81631511 6.04928311 6.94771226 7.2176242
 6.61427325 7.57168503 6.76656132 6.23255435 6.77662643 7.18201412
 6.78189108 7.5299813  6.65903808 6.93395891 5.8793463  7.35845169
 7.41504061 7.14609464 6.40335515 6.92546973 6.40949607 7.82626061
 6.31873931 7.27907227 6.75600499 6.455489   6.94796232 7.2906283
 7.28843425 6.56008644 7.24665727 6.9227526  6.729867   6.

In [164]:
final_predict_exp = np.exp(final_predict)

In [165]:
final_predict_exp

array([1363.35376887,  392.75206699,  392.3317246 , 1192.31250503,
        424.35380268,  972.36072127,  666.53594029, 1545.37561689,
       1631.37130597, 1198.80946767,  592.28810091, 1041.10384603,
       1461.58490517,  512.26959254,  860.57976407, 1041.60463397,
       1333.31194784, 2242.88641321,  612.69012547,  564.0935519 ,
       1249.00283415,  608.76289628,  695.30708233,  710.47708024,
        418.50402388, 2157.20162873,  597.25533084,  549.09765895,
        582.24922352, 1198.80946767,  938.92311199,  997.4100242 ,
        981.94390246,  798.8961122 , 1381.00099813, 1362.95504993,
        938.91271656, 1333.06886541,  522.6484515 , 2108.60633804,
       1972.72233915, 1198.80946767,  303.2895553 , 1569.40475192,
       1041.73711308, 1994.6068276 , 1032.43556381, 1971.30071223,
       1042.36600528, 1277.36927487, 1194.51213586,  648.73413693,
        779.82089372, 1295.12656688,  357.62378871,  970.37479074,
        912.61591349,  423.80909728, 1040.76600337, 1363.24641

IMPORTANTE: APLICAR LO MISMO A ESTOS DATOS QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 391 FILAS, SI O SI

**Entonces, si al cargar los datos de train usé `index_col=0` para que utilizara la primera columna del conjunto de datos como índice, ¿tendré que hacerlo también para el conjunto `test.csv`?**

In [ ]:
# ¿Qué opináis?
# Sí

![](https://www.tierraljelechu.com/web/img/wiki_up/1.996-SorpresaDto.-1-Red.jpg)

### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle. 

**¿Qué es lo que subirás a Kaggle?**

In [ ]:
predictions_submit = model.predict(X_pred)
predictions_submit

array([1097.6423004 , 1095.00755928, 1084.79793741, 1101.01825451,
        981.53500887, 1062.70472731, 1123.17875843, 1136.91660235,
       1100.68891187, 1171.05845611, 1061.49713762, 1039.67799563,
       1120.5592512 , 1084.55112248, 1067.42530516, 1144.60126397,
       1008.086748  , 1087.21311678, 1117.59516743, 1166.6672209 ,
       1136.28517025, 1137.01114934, 1201.1384173 , 1195.5395924 ,
       1155.57935199, 1095.22712104, 1205.96877603, 1141.07304191,
       1087.734768  , 1145.58929189, 1069.84048452, 1117.48538655,
       1169.08240026, 1183.02457206, 1166.99656354, 1048.59589281,
       1143.28389341, 1069.57766753, 1078.759989  , 1072.74768646,
       1196.62216731, 1063.58297435, 1030.59182845,  995.79128941,
       1065.55903019, 1129.78084514, 1071.26763597, 1167.64001493,
       1224.39673002, 1248.32896191, 1181.26807797, 1092.56512673,
       1090.69885177, 1189.17230135, 1066.32749635, 1158.8727784 ,
       1102.11606331, 1053.33170455, 1106.66997846, 1072.30856

**¡PERO! Para subir a Kaggle la predicción, ésta tendrá que tener una forma específica y no valdrá otra.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [172]:
sample = pd.read_csv("data/sample_submission.csv")

In [173]:
sample.head()

,laptop_ID,Price_euros
0,750,500
1,726,500
2,633,500
3,363,650
4,319,650


In [154]:
sample.shape

(391, 2)

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [175]:
submission = pd.DataFrame({"laptop_ID": sample['laptop_ID'], "Price_euros": final_predict_exp})

In [176]:
submission.head()

,laptop_ID,Price_euros
0,750,1363.353769
1,726,392.752067
2,633,392.331725
3,363,1192.312505
4,319,424.353803


In [177]:
submission.shape

(391, 2)

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [179]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")


In [180]:
chequeator(submission)

You're ready to submit!
